## Recommendation System
Collaborative filtering with implicit feedback based on latent factors. Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [62]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.mllib.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

import pandas as pd
import numpy as np

from test_model import (get_patent_fields_list, get_ml_patents, 
                        create_title_abstract_col,trim_data, 
                        structure_dataframe, partition_dataframe, 
                        build_pipeline, process_docs, pat_inv_map, get_topics)
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary, mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import requests
from bs4 import BeautifulSoup
import pickle
import math

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

In [64]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Data understanding - Acquire data

In [296]:
# load pickled dataset
with open('/Users/lee/Documents/techniche/techniche/data/raw_data_1000', 'rb') as f:
    raw_data_1000 = pickle.load(f)

In [297]:
# define desired keys/columns as criteria to subset dataset
retained_keys = ['patent_number', 'patent_firstnamed_assignee_id']

In [298]:
# subset raw dataset by desired keys/columns
data_1000 = trim_data(data=raw_data_1000, keys=retained_keys)

In [299]:
# create Pandas dataframe
df_1000 = pd.DataFrame(data_1000)

### Data preparation

In [300]:
# create new rating column and assign value of 1
df_1000['rating'] = 1

In [301]:
# drop row that contains invalid data in patent_number column
df_1000[df_1000.patent_number.str.contains('[RE]')]
df_1000 = df_1000.drop(df_1000.index[[717]])

In [302]:
# drop NaNs in patent_firstnamed_assignee_id column
df_1000.info()
df_1000 = df_1000.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 999
Data columns (total 3 columns):
patent_firstnamed_assignee_id    972 non-null object
patent_number                    999 non-null object
rating                           999 non-null int64
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


In [303]:
# convert patent_number column from string to int
df_1000 = df_1000.astype({'patent_number': 'int64'})
# uncomment to confirm
# df_1000.info()

In [107]:
# convert alphanumeric patent_firstnamed_assignee_id col to int
df_1000 = df_1000.astype({'patent_number': 'int64'})
# s = 'org_VU2IXnxgxGIK8A8oQrwm'

# code = [ord(c) for c in s]
# code

In [308]:
df_1000.head(3)

,patent_firstnamed_assignee_id,patent_number,rating
0,org_VU2IXnxgxGIK8A8oQrwm,10226194,1
1,org_9cmRc2rH8nbl8O9VuxYL,10228278,1
2,org_8O8xQifxyiW5pZB2KuDx,10228693,1


In [307]:
def alphanum_to_int(string):
    return int.from_bytes(string.encode(), 'little')

In [311]:
num_1 = alphanum_to_int('org_VU2IXnxgxGIK8A8oQrwm')
num_1

2684112930335669204981933253220615680031947592678911734383

In [312]:
num_2 = alphanum_to_int('org_9cmRc2rH8nbl8O9VuxYL')
num_2

1872084152957228981429292242778804786867770061336955286127

In [ ]:
def int_to_alphanum(string):
    return n.to_bytes(math.ceil(n.bit_length() / 8), 'little').decode()

In [83]:
df_1000 = df_1000.apply(pd.to_numeric)

ValueError: ('Unable to parse string "org_VU2IXnxgxGIK8A8oQrwm" at position 0', 'occurred at index patent_firstnamed_assignee_id')

In [ ]:
s = "4525ABT2"
table = str.maketrans('ABCDEFGHIJKLMNOPQRSTUVWXYZ',
                          '22233344455566677778889999')
s.translate(table)
'45252282'

In [71]:
sp_df_1000 = spark.createDataFrame(df_1000)

In [72]:
sp_df_1000.show()

+-----------------------------+-------------+------+
|patent_firstnamed_assignee_id|patent_number|rating|
+-----------------------------+-------------+------+
|         org_VU2IXnxgxGIK8...|     10226194|     1|
|         org_9cmRc2rH8nbl8...|     10228278|     1|
|         org_8O8xQifxyiW5p...|     10228693|     1|
|         org_q9Bn28RHhpYrQ...|     10228922|     1|
|         org_EilEWQcC6UiqH...|     10228931|     1|
|         org_ID497r4tFbCIa...|     10229106|     1|
|         org_p6ofWD2xFNSny...|     10229109|     1|
|         org_Vbc6obpnxWM42...|     10229113|     1|
|         org_uSkGGmX0kIBgx...|     10229148|     1|
|         org_q9Bn28RHhpYrQ...|     10229156|     1|
|         org_p6ofWD2xFNSny...|     10229173|     1|
|         org_q9Bn28RHhpYrQ...|     10229187|     1|
|         org_q9Bn28RHhpYrQ...|     10229189|     1|
|         org_iwO2oOJ6VIBd9...|     10229357|     1|
|         org_q9Bn28RHhpYrQ...|     10229368|     1|
|         org_9D8x1qL3IRASp...|     10229673| 

In [81]:
sp_df_1000.dtypes

[('patent_firstnamed_assignee_id', 'string'),
 ('patent_number', 'string'),
 ('rating', 'bigint')]

In [73]:
sp_df_1000.limit(5).toPandas()

,patent_firstnamed_assignee_id,patent_number,rating
0,org_VU2IXnxgxGIK8A8oQrwm,10226194,1
1,org_9cmRc2rH8nbl8O9VuxYL,10228278,1
2,org_8O8xQifxyiW5pZB2KuDx,10228693,1
3,org_q9Bn28RHhpYrQjKvraAH,10228922,1
4,org_EilEWQcC6UiqHcSGx9mb,10228931,1


In [21]:
# TODO (Lee) - from topic_model - convert dataframe from subsetted dict, organize columns and sort by patent_date
# df_1000 = structure_dataframe(data=data_1000)

In [ ]:
# partition df_1000 into train and test dataframes
# data_train_1000, data_test_1000 = partition_dataframe(df_1000, .8)

In [ ]:
df = spark.read.csv(data_1000)

In [33]:
df = spark.read.load("/Users/lee/Documents/techniche/techniche/data/raw_data_1000",
                     format="json")

Py4JJavaError: An error occurred while calling o117459.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 13, localhost, executor driver): java.io.CharConversionException: Invalid UTF-32 character 0x3031392d(above 10ffff)  at char #1, byte #7)
	at com.fasterxml.jackson.core.io.UTF32Reader.reportInvalid(UTF32Reader.java:189)
	at com.fasterxml.jackson.core.io.UTF32Reader.read(UTF32Reader.java:150)
	at com.fasterxml.jackson.core.json.ReaderBasedJsonParser.loadMore(ReaderBasedJsonParser.java:153)
	at com.fasterxml.jackson.core.json.ReaderBasedJsonParser._skipWSOrEnd(ReaderBasedJsonParser.java:2017)
	at com.fasterxml.jackson.core.json.ReaderBasedJsonParser.nextToken(ReaderBasedJsonParser.java:577)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1$$anonfun$apply$3.apply(JsonInferSchema.scala:56)
	at org.apache.spark.sql.catalyst.json.JsonInferSchem

In [ ]:
df.persist()

In [ ]:
df.limit(5).toPandas()

### Data preparation
Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [ ]:
latent_ratings = 

In [ ]:
# Columns are: assigneeId, itemId, (transaction x confidence)

In [ ]:
#### Partition data into training and test sets
(training, test) = latent_ratings.randomSplit([0.8, 0.2])

### Model # 1

In [12]:
# set implicitPrefs to True to get better results b/c latent_ratings matrix 
# derived from another source of information (i.e. it is inferred from other signals), 

In [79]:
# split into 
(training, test) = sp_df_1000.randomSplit([0.8, 0.2])

In [80]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5,rank=4, 
          regParam=0.01, 
          userCol="userId", 
          itemCol="movieId", 
          ratingCol="rating",
          coldStartStrategy="drop")

TypeError: object() takes no parameters

In [ ]:
# fit the ALS model to the training set
model = als.fit(training)

In [ ]:
# build recommendation model using ALS
rank = 10
numIterations = 10
alpha=

# als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True,
#           userCol="userId", itemCol="movieId", ratingCol="rating")

In [ ]:
# build recommendation model using ALS based on implicit ratings
als = ALS(maxIter=5,
          rank=4, 
          regParam=0.01, 
          userCol="patent_firstnamed_assignee_id", 
          itemCol="patent_number", 
          ratingCol="new",
          coldStartStrategy="True")

In [75]:
# second example
model = ALS.trainImplicit(sp_df_1000, numIterations, alpha=0.01)

NameError: name 'numIterations' is not defined

#### Model #1 - Evaluation - Compare to naive baseline
Compare model evaluation result with naive baseline model that only outputs (for explicit - the average rating (or you may try one that outputs the average rating per movie).

#### Model #1 - Optimize model

In [76]:
als_model =  ALS(userCol="patent_firstnamed_assignee_id",
                 itemCol="patent_number",
                 ratingCol="rating", 
                 coldStartStrategy="drop")             

TypeError: object() takes no parameters

In [78]:
als = ALS(maxIter=5, regParam=0.01, 
          userCol="patent_firstnamed_assignee_id", 
          itemCol="patent_number", 
          ratingCol="rating")

TypeError: object() takes no parameters

In [ ]:
model = als.fit(sp_df_1000)

In [ ]:
params = ParamGridBuilder().addGrid(als_model.regParam, [0.01,0.001,0.1]).addGrid(als_model.rank, [4,10,50]).build()


## instantiate crossvalidator estimator
cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(movie_ratings)    

In [ ]:
# Getting Predictions for a New User

In [ ]:
predictions = model.transform(test)